---
### The main objective of this notebook is to use the visualization library, Folium, to visualize the results.

We finally visualise clinics with negative reviews by top topics (issues) discussed and may be able to group clinics into clusters

---

In [21]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
import folium # plotting library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [22]:
df = pd.read_csv("data/review_sentiment_topics.csv")

In [23]:
df.shape

(18, 6)

In [24]:
address = 'Singapore'

geolocator = Nominatim(user_agent="Your_app-name")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


#### Map 1:
Show all clinics and popup top 3 topics discussed in the Google reviews with Sentiment

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

for lat, lng, label in zip(df.lat, df.lng, df.TopTopics):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

 ![map screenshot for git](data/img_1.png)

#### Map 2:
Split clinics into two clusters:
1. Cluster with Long Waiting time
2. Cluster without Long Waiting time

In [26]:
df['cluster'] = df.TopTopics.apply(lambda x: 1 if 'Long queue/wait time' in str(x) else 2)

In [27]:
df['cluster_label'] = df.cluster.apply(lambda x: 'Long queue/wait time' if int(x)==1 else 'NO Long queue/wait time')

In [28]:
df.head(3)

,place_id,TopicSentiment,reviews,TopTopics,lat,lng,cluster,cluster_label
0,ChIJDQ5i_q892jEREshja_0JOeU,['Doctor/Staff/Receptionist behavior_positive'...,"Came here twice. Staff at the dental side, if ...",['Doctor/Staff/Receptionist behavior_positive'...,1.372618,103.949543,2,NO Long queue/wait time
1,ChIJL7ZNaFQ82jERG0nE6vQksRM,"['Long queue/wait time_positive', 'Doctor/Staf...",Dr George is one of the best Dr that listened ...,['Doctor/Staff/Receptionist behavior_positive'...,1.366480,103.964491,1,Long queue/wait time
2,ChIJL_NVReMV2jERUxYmGmVoKOQ,"['Long queue/wait time_negative', 'Rude Custom...",Their queue system sucks . Bottleneck is not r...,"['Long queue/wait time_negative', 'Service qua...",1.406028,103.902254,1,Long queue/wait time


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(2)
ys = [i+x+(i*x)**2 for i in range(2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#rainbow = ["#FF33F6","#33E0FF","#7A33FF","#FF7D33","#7E0548","#2DBB1F","#05127E"]
# add markers to the map
markers_colors = []
for lat, lon, label, cluster in zip(df['lat'], df['lng'], df['cluster_label'], df['cluster']):
    label = folium.Popup(str(label), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

 ![map screenshot for git](data/img_3.png)

As can be seen Raffles clinics at Punggol, Sengkang, Hougang --> users reported Long queue/wait time. On the other hand Raffles clinics at Tampines, Simei and Bedok --> user do not report Long queue/wait time. Should be noted multiple clinics can be at the same address. For instance at Pasir Ris MRT there are two clinics GP and Dental.